In [1]:
import cvxpy as cp
import numpy as np
import random

## 1. Make simulate data

make matched point pairs with noise

In [33]:
def Make_Simulate_Data(num, groud_truth_T, noise_w = 0.01):
    cloud_1 = np.random.rand(num, 3)
    R = groud_truth_T[0:3, 0:3]
    t = groud_truth_T[0:3, 3]
    matrix_Cs = [np.eye(3) for i in range(num)]
    
    cloud_2_t = np.dot(R, np.transpose(cloud_1)) + np.reshape(t, [3,1])
    cloud_2 = cloud_2_t.transpose() + np.random.rand(num, 3) * noise_w
    return cloud_1, cloud_2, matrix_Cs

In [34]:
ground_truth = np.array(
[[ 0.99997614, -0.00646622, -0.00232642, 1.32194518],
 [ 0.00646101,  0.99997672, -0.0022425, 0.01495234],
 [ 0.00234087,  0.00222741,  0.99999442, 0.01125935],
 [0.0,          0.0,         0.0,       1.0]]
)
cloud_1, cloud_2, matrix_Cs = Make_Simulate_Data(100, ground_truth, 0.05)
print('Generate ', cloud_1.shape[0], ' point pairs.')

Generate  100  point pairs.


## 2. Calculate matrix M

In [51]:
def CalculateMatrixM(cloud_1, cloud_2, matrix_Cs):
    # corresponding variable is : [vec(R).t, t.t, 1].t
    M = np.zeros([13, 13])
    for i in range(cloud_1.shape[0]):
        N_i = np.concatenate((cloud_1[i,0]*np.eye(3), cloud_1[i,1]*np.eye(3), 
                              cloud_1[i,2]*np.eye(3), np.eye(3), -cloud_2[i,:].reshape([3,1])), axis = 1)
        M = M + np.dot(N_i.transpose(), np.dot(matrix_Cs[i], N_i))
    return M

def CalculateMatrixReOrderM(cloud_1, cloud_2, matrix_Cs):
    # corresponding variable ouput : [vec(R).t, 1, t.t].t
    M = np.zeros([13, 13])
    for i in range(cloud_1.shape[0]):
        N_i = np.concatenate((cloud_1[i,0]*np.eye(3), cloud_1[i,1]*np.eye(3), 
                              cloud_1[i,2]*np.eye(3), -cloud_2[i,:].reshape([3,1]), np.eye(3)), axis = 1)
        M = M + np.dot(N_i.transpose(), np.dot(matrix_Cs[i], N_i))
    return M

def MarginalizeReOrderM(M):
    M_r_r = M[0:10, 0:10]
    M_r_t = M[0:10, 10:13]
    M_t_r = M[10:13, 0:10]
    M_t_t = M[10:13, 10:13]
    
    M_t_t_inv = np.linalg.inv(M_t_t)
    Q = M_r_r - np.dot(M_r_t, np.dot(M_t_t_inv, M_t_r))
    return Q, M_t_t_inv, M_t_r

In [52]:
M = CalculateMatrixReOrderM(cloud_1, cloud_2, matrix_Cs)
Q, M_t_t_inv, M_t_r = MarginalizeReOrderM(M)

## 3. Form the duality

# Other test

In [1]:
from kdtree import *

import open3d as o3d
from pyntcloud import PyntCloud

In [2]:
def feature_extraction_fpfh(cloud_numpy, voxel_size = 0.05):
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(cloud_numpy[:,0:3])
    
    point_cloud_o3d = point_cloud_o3d.voxel_down_sample(voxel_size) 
    
    radius_normal = voxel_size * 2 
    point_cloud_o3d.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    point_cloud_o3d_fpfh = o3d.registration.compute_fpfh_feature(
        point_cloud_o3d,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return point_cloud_o3d, point_cloud_o3d_fpfh

In [3]:
def feature_based_icp(cloud_1, cloud_2, voxel_size = 0.05):
    source_cloud, source_feature = feature_extraction_fpfh(cloud_2, voxel_size)
    target_cloud, target_feature = feature_extraction_fpfh(cloud_1, voxel_size)
    
    distance_threshold = voxel_size * 1.5 
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_cloud, target_cloud, source_feature, target_feature, o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [32]:
def load_cloud(path):
    point_cloud_pynt = PyntCloud.from_file(path)
    point_cloud_o3d = point_cloud_pynt.to_instance("open3d", mesh=False)
    cloud = np.asarray(point_cloud_o3d.points)[:,0:3]
    return cloud

def load_cloud_and_color(path):
    point_cloud_pynt = PyntCloud.from_file(path)
    point_cloud_o3d = point_cloud_pynt.to_instance("open3d", mesh=False)
    cloud = np.asarray(point_cloud_o3d.points)[:,0:3]
    return cloud, point_cloud_o3d.colors

def Transform_cloud(cloud, R, t):
    new_cloud = np.dot(R, np.transpose(cloud)) + np.reshape(t, [3,1])
    return new_cloud.transpose()

In [24]:
# randomly sample point in cloud second
def find_match(root_cloud_first, points_first, cloud_second, ratio = 0.1, distance_threshold = 1.0):
    #root_cloud_first = kdtree_construction(points_first, leaf_size=4)
    matched_first = []
    matched_second = []
    k = 1
    
    #distance_threshold = 1.0
    subsample_ids = np.random.choice(cloud_second.shape[0], int(cloud_second.shape[0]*ratio), replace=False)
    distance_sum = 0
    for idx in subsample_ids:
        query = np.asarray(cloud_second[idx,:])
        result_set = KNNResultSet(capacity=k)
        kdtree_knn_search(root_cloud_first, points_first, result_set, query)
        if(result_set.dist_index_list[0].distance > distance_threshold):
            continue
        match_id = result_set.dist_index_list[0].index
        matched_first.append(points_first[match_id,:])
        matched_second.append(query)
        #print(result_set.dist_index_list[0].distance)
        distance_sum += result_set.dist_index_list[0].distance
        
    distance_sum = distance_sum / len(matched_first)
    return np.asarray(matched_first), np.asarray(matched_second), distance_sum

def SvdBasedPoseEstimation(matched_1, matched_2):
    centroid_1 = matched_1.mean(axis=0)
    centroid_2 = matched_2.mean(axis=0)
    
    centered_1 = matched_1 - centroid_1
    centered_2 = matched_2 - centroid_2
    
    Matrix_W = np.dot(np.transpose(centered_1), centered_2)
    U,eigenvalues,VT = np.linalg.svd(Matrix_W)
    #eigenvectors = np.transpose(VT)
    R = np.dot(U, VT)
    t = np.transpose(centroid_1) - np.dot(R, np.transpose(centroid_2))
    return R, t

def ICP(points_first, points_second, iteration = 10, ratio = 0.1, max_distance = 1.0, verbose = True):    
    result_R = np.eye(3)
    result_t = np.zeros(3)
    points_second_transformed = Transform_cloud(points_second[:,0:3], result_R, result_t)
    root_cloud_first = kdtree_construction(points_first, leaf_size=4)
    for i in range(iteration):
        if(verbose):
            print('--> iteration : ', i)
            
        if(i > iteration*2/3):
            max_distance_t = max_distance/3
        elif(i > iteration/2):
            max_distance_t = max_distance/2
        else:
            max_distance_t = max_distance
        matched_first, matched_second, distance_average = find_match(root_cloud_first, points_first[:,0:3], points_second_transformed, ratio, max_distance_t)
        if(verbose):
            print('    matches : ', matched_first.shape[0], ' average distance : ',distance_average)
        if(distance_average < 0.05):
            if(verbose):
                print(' Converged !')
            break
        R, t = SvdBasedPoseEstimation(matched_first, matched_second)
        result_R = np.dot(R, result_R)
        result_t = np.dot(R, result_t) + t
        points_second_transformed = Transform_cloud(points_second_transformed, R, t)
        
    return points_second_transformed, result_R, result_t

In [29]:
def inverse_T(T):
    Final_T = np.eye(4,4)
    inverse_R = np.transpose(T[0:3, 0:3])
    inverse_t = -np.dot(inverse_R, T[0:3, 3])
    Final_T[0:3, 0:3] = inverse_R
    Final_T[0:3, 3] = inverse_t
    return Final_T

def rotationMatrixToEulerAngles(R):
    sy = np.sqrt(R[0,0] * R[0,0] +R[1,0] * R[1,0] );
    singular = sy < 1e-6;
    if (not singular) :
        x = np.arctan2(R[2,1] , R[2,2]);
        y = np.arctan2(-R[2,0], sy);
        z = np.arctan2(R[1,0], R[0,0]);
    else :
        x = np.arctan2(-R[1,2], R[1,1]);
        y = np.arctan2(-R[2,0], sy);
        z = 0;
    #if 1
    x = x*180.0/3.141592653589793;
    y = y*180.0/3.141592653589793;
    z = z*180.0/3.141592653589793;
    #endif
    return np.array([x, y, z]);

In [6]:
def FPFH_and_ICP(cloud_1, cloud_2, iteration = 10, verbose = True):
    FPFH_result = feature_based_icp(cloud_1, cloud_2, 0.2)
    result_R = FPFH_result.transformation[0:3, 0:3]
    result_t = FPFH_result.transformation[0:3, 3]
    points_second_transformed = Transform_cloud(cloud_2, result_R, result_t)
    points_second_transformed, R, t = ICP(cloud_1, points_second_transformed, iteration, 0.1, 3.0, verbose)
    result_R = np.dot(R, result_R)
    result_t = np.dot(R, result_t) + t
    return points_second_transformed, result_R, result_t

#### Initialize with FPFH

In [17]:
points_first = load_cloud("D:/SFM/winter_garden/garden_l_sub_2.ply")
points_second = load_cloud("D:/SFM/winter_garden/sfm_sub_garden.ply") * 5
FPFH_result = feature_based_icp(points_first, points_second, 0.05)
result_R = FPFH_result.transformation[0:3, 0:3]
result_t = FPFH_result.transformation[0:3, 3]
points_second_transformed = Transform_cloud(points_second, result_R, result_t)

#### Initialize with human

In [18]:
human_guess = np.array([
[1.974505543709, -0.080609202385, -2.005241155624, 0.632752060890],
[2.001228809357, -0.131519153714, 1.975841164589, 7.528551101685],
[-0.150247544050, -2.811114788055, -0.034939885139, 1.992925643921]
])
Rs = human_guess[0:3, 0:3]
scale_guess = np.sqrt(np.dot(np.transpose(Rs), Rs)[0,0])

result_R = human_guess[0:3, 0:3] / scale_guess
result_t = human_guess[0:3, 3]
print("the initial scale guess : ", scale_guess)

the initial scale guess :  2.8153442443209435


In [35]:
points_first = load_cloud("D:/SFM/winter_garden/garden_l_sub_1.ply")
points_second, colors_second = load_cloud_and_color("D:/SFM/winter_garden/fused_5.ply")
points_second = points_second * scale_guess
points_second_transformed = Transform_cloud(points_second, result_R, result_t)

In [26]:
points_second_transformed, R, t = ICP(points_first, points_second_transformed, 20, 0.01, 3.0, True)

--> iteration :  0
    matches :  8125  average distance :  0.3039414972240154
--> iteration :  1
    matches :  8126  average distance :  0.26273079585021164
--> iteration :  2
    matches :  8124  average distance :  0.2316590798277032
--> iteration :  3
    matches :  8122  average distance :  0.20945110889996982
--> iteration :  4
    matches :  8124  average distance :  0.1915288055260187
--> iteration :  5
    matches :  8122  average distance :  0.17773031326152436
--> iteration :  6
    matches :  8124  average distance :  0.16751799101647438
--> iteration :  7
    matches :  8125  average distance :  0.16411750730540298
--> iteration :  8
    matches :  8121  average distance :  0.1574458172599271
--> iteration :  9
    matches :  8121  average distance :  0.154443245908905
--> iteration :  10
    matches :  8124  average distance :  0.15394604818500793
--> iteration :  11
    matches :  8112  average distance :  0.15095918575806255
--> iteration :  12
    matches :  8116  ave

In [39]:
Final_R = np.dot(R, result_R)
Final_t = np.dot(R, result_t) + t
Final_T = np.eye(4,4)
Final_T[0:3, 0:3] = Final_R
Final_T[0:3, 3] = Final_t
print("Final Matrix is : ")
print(Final_T)
print("Inverse T is : \n", inverse_T(Final_T))
print("Euler angles are : ", rotationMatrixToEulerAngles(inverse_T(Final_T)[0:3, 0:3]))
print("Should copy the Final matrix to unity.")

Final Matrix is : 
[[ 0.71697014  0.00971432 -0.69703609  0.67756739]
 [ 0.69699781 -0.02743195  0.71654827  7.25156145]
 [-0.01216027 -0.99957636 -0.02643874  1.54380928]
 [ 0.          0.          0.          1.        ]]
Inverse T is : 
 [[ 0.71697014  0.69699781 -0.01216027 -5.52134488]
 [ 0.00971432 -0.02743195 -0.99957636  1.73549765]
 [-0.69703609  0.71654827 -0.02643874 -4.68298854]
 [ 0.          0.          0.          1.        ]]
Euler angles are :  [92.11310482 44.18969387  0.77626023]
Should copy the Final matrix to unity.


In [38]:
point_cloud_o3d_1 = o3d.geometry.PointCloud()
point_cloud_o3d_1.points = o3d.utility.Vector3dVector(points_first[:,0:3])
colors = [[1, 0, 0] for i in range(points_first.shape[0])]
point_cloud_o3d_1.colors = o3d.utility.Vector3dVector(colors)

point_cloud_o3d_2 = o3d.geometry.PointCloud()
points_second_transformed = Transform_cloud(points_second, Final_R, Final_t)
point_cloud_o3d_2.points = o3d.utility.Vector3dVector(points_second_transformed[:,0:3])
point_cloud_o3d_2.colors = o3d.utility.Vector3dVector(np.asarray(colors_second) / 255)

o3d.visualization.draw_geometries([point_cloud_o3d_1, point_cloud_o3d_2])

![image](images/fused_5_matched.PNG)